In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='d60005d1-fec1-4717-a69c-59ef0b3087a1', project_access_token='p-50d537da9be078d7d5ab2372ab67d98f6ba79f18')
pc = project.project_context

In [2]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import types
from botocore.client import Config
import ibm_boto3

In [3]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_395cc1a0b7654fbba69323d9879cfeb3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='49ijI91qmQJR3Lvcia1jOFqTujDLUdwLaZx3MCMQ2Uox',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_395cc1a0b7654fbba69323d9879cfeb3.get_object(Bucket='titanic-donotdelete-pr-dahl3novhxbeev',Key='train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.drop(['Name','Ticket','Fare','Cabin','Embarked'], axis=1, inplace=True)
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 7 columns):
PassengerId    714 non-null int64
Survived       714 non-null int64
Pclass         714 non-null int64
Sex            714 non-null object
Age            714 non-null float64
SibSp          714 non-null int64
Parch          714 non-null int64
dtypes: float64(1), int64(5), object(1)
memory usage: 44.6+ KB


In [5]:
df.set_index('PassengerId', inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,,
1,0,3,male,22.0,1,0
2,1,1,female,38.0,1,0
3,1,3,female,26.0,0,0
4,1,1,female,35.0,1,0
5,0,3,male,35.0,0,0


In [6]:
df['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,,
1,0,3,0,22.0,1,0
2,1,1,1,38.0,1,0
3,1,3,1,26.0,0,0
4,1,1,1,35.0,1,0
5,0,3,0,35.0,0,0


In [7]:
Feature = df[['Pclass','Sex','Age','SibSp','Parch']]
Feature.head()

,Pclass,Sex,Age,SibSp,Parch
PassengerId,,,,,
1,3,0,22.0,1,0
2,1,1,38.0,1,0
3,3,1,26.0,0,0
4,1,1,35.0,1,0
5,3,0,35.0,0,0


In [8]:
y=df['Survived'].values
y[0:5]

array([0, 1, 1, 1, 0])

In [9]:
X=Feature
X= preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app


array([[ 0.91123237, -0.75905134, -0.53037664,  0.52457013, -0.50589515],
       [-1.47636364,  1.31743394,  0.57183099,  0.52457013, -0.50589515],
       [ 0.91123237,  1.31743394, -0.25482473, -0.55170307, -0.50589515],
       [-1.47636364,  1.31743394,  0.36516706,  0.52457013, -0.50589515],
       [ 0.91123237, -0.75905134,  0.36516706, -0.55170307, -0.50589515]])

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (571, 5) (571,)
Test set: (143, 5) (143,)


KNN

In [11]:
Ks = 12
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
ConfustionMx = [];
for n in range(1,Ks):
    
    #Train Model and Predict  
    neigh = KNeighborsClassifier(n_neighbors = n).fit(X_train,y_train)
    yhat=neigh.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)

    
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])

print(mean_acc)
print ('Best k is ', np.where(mean_acc==max(mean_acc)))

[0.72727273 0.76223776 0.83916084 0.81818182 0.81818182 0.8041958
 0.8041958  0.81818182 0.81118881 0.79020979 0.7972028 ]
Best k is  (array([2]),)


In [12]:
k = 3
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
final_neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat_KN = final_neigh.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat_KN))
print ("Jaccard %.2f" % jaccard_similarity_score(y_test, yhat_KN))
print ("f1_score %.2f" % f1_score(y_test, yhat_KN, average='weighted'))

Train set Accuracy:  0.8739054290718039
Test set Accuracy:  0.8391608391608392
Jaccard 0.84
f1_score 0.84


Decision Tree

In [13]:
from sklearn.tree import DecisionTreeClassifier
md = 12
mean_acc = np.zeros((md-1))
std_acc = np.zeros((md-1))
ConfustionMx = [];
for n in range(1,md):
 
    Tree = DecisionTreeClassifier(criterion="entropy", max_depth = n).fit(X_train,y_train)
    yhat=Tree.predict(X_test)
    mean_acc[n-1] = metrics.accuracy_score(y_test, yhat)
    std_acc[n-1]=np.std(yhat==y_test)/np.sqrt(yhat.shape[0])
print (mean_acc)
print ('Best n is ', np.where(mean_acc==max(mean_acc)))

[0.77622378 0.76923077 0.77622378 0.7972028  0.83216783 0.76223776
 0.83216783 0.7972028  0.81818182 0.81818182 0.81818182]
Best n is  (array([4, 6]),)


In [14]:
final_Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 5).fit(X_train,y_train)
yhat_Tree=final_Tree.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_Tree.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat_Tree))
print ("Jaccard %.2f" % jaccard_similarity_score(y_test, yhat_Tree))
print ("f1_score %.2f" % f1_score(y_test, yhat_Tree, average='weighted'))

Train set Accuracy:  0.8528896672504378
Test set Accuracy:  0.8251748251748252
Jaccard 0.83
f1_score 0.82


Support Vector Machine

In [15]:
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
str=''
a=('linear', 'poly', 'rbf', 'sigmoid')
mean_accc=[]
for str in a:
    clf = svm.SVC(kernel=str)
    clf.fit(X_train, y_train)
    yhat = clf.predict(X_test)
    print(str,'-',metrics.accuracy_score(y_test, yhat))

linear - 0.7762237762237763
poly - 0.7762237762237763
rbf - 0.8251748251748252
sigmoid - 0.6923076923076923


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [16]:
final_clf = svm.SVC(kernel='rbf')
final_clf.fit(X_train, y_train)
yhat_clf = final_clf.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(y_train, final_clf.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, yhat_clf))
print ("Jaccard %.2f" % jaccard_similarity_score(y_test, yhat_clf))
print ("f1_score %.2f" % f1_score(y_test, yhat_clf, average='weighted'))

Train set Accuracy:  0.8353765323992994
Test set Accuracy:  0.8251748251748252
Jaccard 0.83
f1_score 0.82


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Logistic Regression

In [17]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=4)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
Solvers=('liblinear', 'sag', 'saga', 'newton-cg', 'lbfgs')
str=''
for str in Solvers:
    LR = LogisticRegression(C=0.01, solver=str).fit(X_train,y_train)
    yhat = LR.predict(X_test)
    print(str,'-',metrics.accuracy_score(y_test, yhat))

liblinear - 0.7902097902097902
sag - 0.7762237762237763
saga - 0.7762237762237763
newton-cg - 0.7762237762237763
lbfgs - 0.7762237762237763


In [18]:
final_LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
yhat_LR = LR.predict(X_test)
print ("Jaccard %.2f" % jaccard_similarity_score(y_test, yhat_LR))
print ("f1_score %.2f" % f1_score(y_test, yhat_LR, average='weighted'))
print ("log loss %.2f" % log_loss(y_test, yhat_LR))

Jaccard 0.78
f1_score 0.77
log loss 7.73


I will use KNN algorithm

In [19]:
import types
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_395cc1a0b7654fbba69323d9879cfeb3 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='49ijI91qmQJR3Lvcia1jOFqTujDLUdwLaZx3MCMQ2Uox',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body_test = client_395cc1a0b7654fbba69323d9879cfeb3.get_object(Bucket='titanic-donotdelete-pr-dahl3novhxbeev',Key='test.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_test = pd.read_csv(body_test)
df_test.drop(['Name','Ticket','Fare','Cabin','Embarked'], axis=1, inplace=True)
df_test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch
0,892,3,male,34.5,0,0
1,893,3,female,47.0,1,0
2,894,2,male,62.0,0,0
3,895,3,male,27.0,0,0
4,896,3,female,22.0,1,1


In [20]:
df_test['Age'].fillna((df_test['Age'].mean()), inplace=True)
df_test['Sex'].replace(to_replace=['male','female'], value=[0,1],inplace=True)
df_test.set_index('PassengerId', inplace=True)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 5 columns):
Pclass    418 non-null int64
Sex       418 non-null int64
Age       418 non-null float64
SibSp     418 non-null int64
Parch     418 non-null int64
dtypes: float64(1), int64(4)
memory usage: 19.6 KB


In [21]:
X_train=df[['Pclass','Sex','Age','SibSp','Parch']]
X_train=preprocessing.StandardScaler().fit(X_train).transform(X_train)
y_train=df['Survived']
X_test=df_test[['Pclass','Sex','Age','SibSp','Parch']]
X_test=preprocessing.StandardScaler().fit(X_test).transform(X_test)
print(X_train[0:5], X_test[0:5], y_train[0:5].values)

[[ 0.91123237 -0.75905134 -0.53037664  0.52457013 -0.50589515]
 [-1.47636364  1.31743394  0.57183099  0.52457013 -0.50589515]
 [ 0.91123237  1.31743394 -0.25482473 -0.55170307 -0.50589515]
 [-1.47636364  1.31743394  0.36516706  0.52457013 -0.50589515]
 [ 0.91123237 -0.75905134  0.36516706 -0.55170307 -0.50589515]] [[ 0.87348191 -0.75592895  0.3349926  -0.49947002 -0.4002477 ]
 [ 0.87348191  1.32287566  1.32553003  0.61699237 -0.4002477 ]
 [-0.31581919 -0.75592895  2.51417495 -0.49947002 -0.4002477 ]
 [ 0.87348191 -0.75592895 -0.25932986 -0.49947002 -0.4002477 ]
 [ 0.87348191  1.32287566 -0.65554483  0.61699237  0.61989583]] [0 1 1 1 0]


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  from ipykernel import kernelapp as app
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


In [22]:
k = 3
final_neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
yhat_KN = final_neigh.predict(X_test)

In [23]:
df_answer=pd.DataFrame(index=df_test.index)
df_answer['Survived']=yhat_KN
df_answer.head(15)

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,0
897,0
898,0
899,0
900,1


In [24]:
from project_lib import Project
project = Project(None,"d60005d1-fec1-4717-a69c-59ef0b3087a1","p-50d537da9be078d7d5ab2372ab67d98f6ba79f18")
test_file = project.get_file("test.csv")
test_file.seek(0)
test_df=pd.read_csv(test_file)
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [25]:
df_answer=pd.DataFrame(index=test_df.index)
df_answer['PassengerId']=test_df['PassengerId']
df_answer['Survived']=yhat_KN
df_answer.head(20)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0


In [26]:
project.save_data("submission.csv", df_answer.to_csv(index=False))

{'file_name': 'submission.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'titanic-donotdelete-pr-dahl3novhxbeev',
 'asset_id': 'a47c6524-0cc4-4485-b26f-e91d844eef9e'}